In [1]:
import json
import sys
sys.path.append("../")
from data import RefineryData
from model import Planning
import pandas as pd

In [2]:
filename = "../examples/ex_2_1.json"

In [3]:
with open(filename, "r") as file:
    data = json.load(file)

In [4]:
ex_data = RefineryData(**data)

In [5]:
ex_data

RefineryData(I=['CM1', 'CM2', 'CM3', 'CM4', 'CM5', 'CM6'], J=['G90', 'G93', 'G97'], K=['SUL', 'RON', 'DON', 'ARW', 'OLE'], Q={'CM1': {'SUL': 0.04, 'RON': 89.5, 'DON': 84.25, 'ARW': 12.7, 'OLE': 39.2}, 'CM2': {'SUL': 0.04, 'RON': 93.2, 'DON': 87.1, 'ARW': 21.4, 'OLE': 32.4}, 'CM3': {'SUL': 0.04, 'RON': 89.8, 'DON': 84.9, 'ARW': 13, 'OLE': 39}, 'CM4': {'SUL': 0.001, 'RON': 54.8, 'DON': 50, 'ARW': 0.05, 'OLE': 0}, 'CM5': {'SUL': 0.001, 'RON': 98.5, 'DON': 93, 'ARW': 70, 'OLE': 0}, 'CM6': {'SUL': 0.001, 'RON': 114, 'DON': 107, 'ARW': 0, 'OLE': 0}}, Pmin={'G90': {'SUL': 0, 'RON': 90.1, 'DON': 85.1, 'OLE': 0, 'ARW': 0}, 'G93': {'SUL': 0, 'RON': 93.1, 'DON': 88.1, 'OLE': 0, 'ARW': 0}, 'G97': {'SUL': 0, 'RON': 97.1, 'DON': 92.1, 'OLE': 0, 'ARW': 0}}, Pmax={'G90': {'SUL': 0.07, 'RON': 999, 'DON': 999, 'OLE': 34, 'ARW': 40}, 'G93': {'SUL': 0.07, 'RON': 999, 'DON': 999, 'OLE': 34, 'ARW': 40}, 'G97': {'SUL': 0.07, 'RON': 999, 'DON': 999, 'OLE': 34, 'ARW': 40}}, LX={'CM1': 2.835, 'CM2': 2.205, 'CM3

In [6]:
ex_model = Planning(ex_data)

In [7]:
results = ex_model.solve()

In [8]:
print("目标值:", results["目标值"])
print("调合配方：")
print(results["调合配方"])
print("产品物性")
print(results["产品物性"])

目标值: 12751.23558551069
调合配方：
          G90       G93       G97
CM1       0.0  2.205703  0.629297
CM2     2.205       0.0       0.0
CM3  3.827387  0.477613       0.0
CM4  0.446248  0.201953       0.0
CM5  1.521365   2.93955  0.939085
CM6       0.0  0.175181  0.274819
产品物性
           G90        G93        G97
SUL   0.030408   0.018442   0.014315
RON  90.439274  93.480569  97.738294
DON       85.1       88.1       92.1
ARW   25.43261       40.0       40.0
OLE  27.588763  17.515078  13.383474


#### 结果分析

##### RON

In [9]:
# TODO: 这里可以添加一些重点突出RON
pd.DataFrame(ex_data.Q)

,CM1,CM2,CM3,CM4,CM5,CM6
SUL,0.04,0.04,0.04,0.001,0.001,0.001
RON,89.50,93.20,89.80,54.800,98.500,114.000
DON,84.25,87.10,84.90,50.000,93.000,107.000
ARW,12.70,21.40,13.00,0.050,70.000,0.000
OLE,39.20,32.40,39.00,0.000,0.000,0.000


In [10]:
pd.DataFrame(ex_data.Pmin)

,G90,G93,G97
SUL,0.0,0.0,0.0
RON,90.1,93.1,97.1
DON,85.1,88.1,92.1
OLE,0.0,0.0,0.0
ARW,0.0,0.0,0.0


CM2的辛烷值(RON)相对来说高一点，希望能把CM2尽量调入到G97

#### 修改条件，找到更合理的解

1. 奖励CM2调入到G97的情况，即目标函数加权重，有可能造经济损失(目标函数变小)
2. 限制CM2调入到G97的量，即限制约束，属于硬约束，有可能使模型不可行
3. 测试CM2调入到G97的最大量，即不考虑经济效益的条件下，满足模型可行的同时最大化CM2对G97的调入量

##### 奖励模型

In [11]:
reward_model = Planning(ex_data)

In [12]:
reward_obj = reward_model.b["CM2","G97"] * 10

In [13]:
obj = reward_model.model.getObjective() + reward_obj

In [14]:
reward_model.model.setObjective(obj,"maximize")

In [15]:
reward_results = reward_model.solve()

In [16]:
print("目标值:", reward_results["目标值"])
print("调合配方：")
print(reward_results["调合配方"])
print("产品物性")
print(reward_results["产品物性"])

目标值: 12764.725213716334
调合配方：
          G90       G93       G97
CM1       0.0     2.835       0.0
CM2  0.856037       0.0  1.348963
CM3  4.074948  0.230052       0.0
CM4  0.584705  0.063496       0.0
CM5   2.48431  2.871452  0.044238
CM6       0.0       0.0      0.45
产品物性
           G90        G93        G97
SUL   0.025039   0.020923   0.029542
RON   90.30742  93.451462  98.405326
DON       85.1       88.1       92.1
ARW  30.653057       40.0  17.341821
OLE  23.332323  20.017336  23.712228


##### 硬约束模型

In [29]:
hard_model = Planning(ex_data)

In [30]:
hard_model.model.addCons(hard_model.b["CM2","G97"] >= 1.34)             # 这边要调右端项

c40

In [31]:
hard_results = hard_model.solve()

In [32]:
print("目标值:", hard_results["目标值"])
print("调合配方：")
print(hard_results["调合配方"])
print("产品物性")
print(hard_results["产品物性"])

目标值: 12751.23558551069
调合配方：
          G90       G93       G97
CM1     2.835       0.0       0.0
CM2       0.0     0.865      1.34
CM3  2.058783  2.246217       0.0
CM4   0.50629   0.14191       0.0
CM5   2.59615  2.746873  0.056978
CM6  0.003777       0.0  0.446223
产品物性
           G90        G93        G97
SUL   0.024857   0.021223   0.029353
RON  90.313405  93.445322  98.399335
DON       85.1       88.1       92.1
ARW  30.565557       40.0  17.721593
OLE  23.928066  19.271411  23.554678


#### 软约束模型

In [33]:
soft_model = Planning(ex_data)

In [34]:
soft_obj = soft_model.b["CM2","G97"]
soft_model.model.setObjective(soft_obj,"maximize")

In [35]:
soft_results = soft_model.solve()

In [36]:
print("目标值:", soft_results["目标值"])
print("调合配方：")
print(soft_results["调合配方"])
print("产品物性")
print(soft_results["产品物性"])

目标值: 1.3489628205644355
调合配方：
          G90       G93       G97
CM1     2.835       0.0       0.0
CM2       0.0  0.856037  1.348963
CM3  1.298904  3.006096       0.0
CM4  0.648201       0.0       0.0
CM5  3.217896  2.137866  0.044238
CM6       0.0       0.0      0.45
产品物性
           G90        G93        G97
SUL   0.021153   0.026104   0.029542
RON  90.357271  93.384994  98.405326
DON       85.1       88.1       92.1
ARW  34.771919  34.508183  17.341821
OLE  20.223655  24.162227  23.712228
